In [5]:
import os, sys
sys.path.insert(0, '/Users/shashank/Workspace/Orgs/Ether/ai-engine/services/recommendation')

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import json as js
from scipy.spatial.distance import cosine

In [8]:
!pip install pydgraph

  Using cached https://files.pythonhosted.org/packages/d5/33/aa0e202e8f69e80fb5e187f5eba8a07d8626683f3fe89f1d268438e10957/pydgraph-2.0.2-py2.py3-none-any.whl
     |████████████████████████████████| 2.3MB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 773kB/s eta 0:00:01


In [9]:
from vectorize import Vectorizer
from lsh import Table, LSH, WordSearch, UserSearch, MinHash
from graph_query import DgraphClient

In [10]:
url = "111.93.155.194:9080"

In [11]:
vectorizer = Vectorizer()
dg = DgraphClient(url=url)

In [12]:
def to_json(data, filename):
    with open(filename + ".json", "w", encoding="utf-8") as f_:
        js.dump(data, f_, ensure_ascii=False, indent=4)

def read_json(json_file):
    with open(json_file) as f_:
        meeting = js.load(f_)
    return meeting

In [13]:
user_kw_query = """
query contextUserInfo($n: string, $t: int) {
  contextUserInfo(func: eq(xid, $n)) @cascade{
    attribute
    xid
    associatedMind {
      name
      type
      xid
    }
    ~hasContext {
    name
    attribute
  }
  hasMeeting (first: $t) {
    xid
    attribute
    hasSegment {
      attribute
      xid
      startTime
      text
      authoredBy {
        attribute
        xid
        name
      }
      hasKeywords {
        values
      }
    }
  }
 }
}
"""

In [14]:
variables = {"$n": "01DJSFMQ5MP8AX83Y89QC6T39E", "$t": "10"}
user_kw_resp = dg.perform_query(query=user_kw_query, variables=variables)

In [15]:
user_kw_resp

{'contextUserInfo': [{'attribute': 'contextId',
   'xid': '01DJSFMQ5MP8AX83Y89QC6T39E',
   'associatedMind': [{'name': 'Software Engineering',
     'type': 'domain',
     'xid': '01DAAQY88QZB19JQZ5PRJFR76Y'}],
   '~hasContext': [{'name': 'platform', 'attribute': 'channelId'}],
   'hasMeeting': [{'xid': 'd2302c88-b832-4ac4-ba4f-10b58721dd2c',
     'attribute': 'instanceId',
     'hasSegment': [{'attribute': 'segmentId',
       'xid': '9d2c9717-12d7-4689-b07f-6d20e1900862',
       'startTime': '2019-10-10T07:03:09Z',
       'text': "So whenever that happened the same level swap also has to it's like a stack right.",
       'authoredBy': {'attribute': 'userId',
        'xid': '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368',
        'name': 'Karthik Muralidharan'},
       'hasKeywords': {'values': ['level swap']}},
      {'attribute': 'segmentId',
       'xid': '606d4e0e-92f1-4a66-a4a5-4b74eea21b4c',
       'startTime': '2019-10-10T06:48:46Z',
       'text': "Meaning when we we like for example aft

In [55]:
def format_response(resp, function_name):
    user_id_dict = {}
    user_kw_dict = {}
    user_kw = []

    for info in resp[function_name]:
        context_obj = info["hasContext"]
        meeting_obj = context_obj["hasMeeting"]
        for m_info in meeting_obj:
            segment_obj = m_info["hasSegment"]
            for segment_info in segment_obj:         
                try:
                    user_id = segment_info.get("authoredBy")["xid"]
                    user_name = segment_info.get("authoredBy")["name"]
                    user_id_dict.update({
                        user_id: user_name
                    })

                    keyword_object = segment_info["hasKeywords"]
                    user_kw.extend(list(set(keyword_object["values"])))
                except Exception as e:
                    print(e)
                    continue
            
                user_kw_dict.update({
                            user_id: user_kw
                        })
    
    return user_kw_dict, user_id_dict

In [56]:
user_kw_dict, user_id_dict = format_response(resp=user_kw_resp, function_name='MLchannelShashankKw')

In [57]:
user_kw_dict, user_id_dict

({'7e7ccbba-232d-411a-a95a-d3f244a35f40': ['fall back on segment wise',
   'domains groups',
   'mechanism or remove',
   'algorithmic side',
   'edge case percentage',
   'noisy and stuff',
   'Keys his level',
   'problem is like meet call entities',
   'candida graphs',
   'kind of intersection',
   'Keys',
   'copy paste',
   'free download',
   'big HR',
   'edge case',
   'big engineering community',
   'Collins audio',
   'Collins',
   'Grass will distort the location',
   'entire stage',
   'Segment ideas',
   'Segment',
   'meeting we get key feature vectors',
   'good approach',
   'key phrases',
   'key phrase service',
   'serialize them and store',
   'data indygraf',
   'recommended Watchers',
   'the entire stage',
   'key feature vectors',
   'Watchers',
   'lot more flexible',
   'context study',
   'query on existing data',
   'Freddy',
   'context instance',
   'figure out a proper',
   'Neptune',
   'mapping uuid',
   'problems with new ideas',
   'uids map',
   'da

In [16]:
ref_user_kw_query = """
query contextUserInfo($n: string, $t: int) {
  contextUserInfo(func: eq(xid, $n)) @cascade{
    attribute
    xid
    associatedMind {
      name
      type
      xid
    }
    ~hasContext {
    name
    attribute
  }
  hasMeeting (first: $t) {
    xid
    attribute
    hasSegment {
      attribute
      xid
      startTime
      text
      authoredBy {
        attribute
        xid
        name
      }
      hasKeywords {
        values
      }
    }
  }
 }
}
"""

In [335]:
user_name_query = """
{
  test(func: eq(xid, $n)){
    name
    xid
    attribute
  }
}
"""

In [68]:
var = {"$n": "01DBB3SN874B4V18DCP4ATMRXA", "$t": "30"}
ref_user_kw_resp = dg.perform_query(query=ref_user_kw_query, variables=var)

In [35]:
ref_user_kw_resp

{'contextUserInfo': [{'attribute': 'contextId',
   'xid': '01DBB3SN874B4V18DCP4ATMRXA',
   'associatedMind': [{'name': 'Software Engineering',
     'type': 'domain',
     'xid': '01DAAQY88QZB19JQZ5PRJFR76Y'},
    {'name': 'Ether Engineering',
     'type': 'custom',
     'xid': '01DAAYHEKY5F4E02QVRJPTFTXV'}],
   '~hasContext': [{'name': 'engineering', 'attribute': 'channelId'}],
   'hasMeeting': [{'xid': '1911f350-0b1e-416a-9990-f074087d7b66',
     'attribute': 'instanceId',
     'hasSegment': [{'attribute': 'segmentId',
       'xid': '166854a3-b107-4fd4-b262-f077e62cd776',
       'startTime': '2019-10-09T09:43:54Z',
       'text': "No, no, no, no, no. Angle that video right now, I have not handled Last-N. And for the time, it what it does it directly links binds of stream to video video tech, you know. Okay. Yeah. So, like No, it's just Ah, well, checking weather of the video is unable for this participant. If Norden it's during the efforts, Anibal capturing the feed.",
       'authore

In [36]:
def format_reference_response(resp, function_name):
    user_dict = {}
    user_kw = []

    for info in resp[function_name]:
        channel_obj = info["~hasContext"]
        meeting_obj = info["hasMeeting"]
        for m_info in meeting_obj:
            segment_obj = m_info["hasSegment"]
            seg_ids = []
            seg_texts = []
            for segment_info in segment_obj: 
                segment_kw = []
                seg_texts.append(segment_info["text"])
                seg_ids.append(segment_info["xid"])
                try:
                    user_id = segment_info.get("authoredBy")["xid"]
                    user_name = segment_info.get("authoredBy")["name"]
                    
                    try:
                        u_id = user_dict[user_id]
                    except KeyError:
                        # Intialize
                        user_dict.update({
                            user_id: {
                                    "name": user_name,
                                    "keywords": None,
                                    "text": seg_texts,
                                    "segment_id": seg_ids
                                }
                            })        

                    keyword_object = segment_info["hasKeywords"]
                    segment_kw.extend(list(set(keyword_object["values"])))
                    
                    user_kw_list = user_dict[user_id].get("keywords")
                    if user_kw_list is not None:
                        user_kw_list.extend(segment_kw)
                        user_dict[user_id].update({
                                "keywords": list(set(user_kw_list))
                            })
                    else:
                        user_dict[user_id].update({
                                "keywords": segment_kw
                            })
                except Exception as e:
                    print(e)
                    continue
    
    return user_dict

In [69]:
ref_user_dict = format_reference_response(resp=ref_user_kw_resp, function_name='contextUserInfo')

In [70]:
len(ref_user_dict.keys())

14

In [39]:
ref_user_dict['62b6ae1d-7f83-4b0b-b205-5f7c72bc3368']['text']

['The best Decay is something like with the weather latest Rebels because we need to rework on our client because just by upgrading the JS library is not working. So we need to spend a little bit more time on that. So I once Parsha finishes on walks it we can you can you can directly find every after like once this is free. I can also look into upgrading that JavaScript. ',
 "Let's create an epic and paste all the discussions. We had one. So we'll just keep track of all the investigations. We do a lot of things, okay. ",
 "So in before post-processing like in the original list that it got it had two more slack Channel slack and slack Channel keywords. So where there was like sea and Earth has capitals and small and everything. So Google those got filtered but these who still came up. So so something like some fun videos that I'm checking on me like an ideal way to remove. ",
 'So the regression that Shri fix like the bug is it already in production? ',
 "Yeah, so sorry said like let's 

In [71]:
var = {"$n": "01DBB3SN99AVJ8ZWJDQ57X9TGX", "$t": "30"}
ml_ref_user_resp = dg.perform_query(query=ref_user_kw_query, variables=var)

In [72]:
ml_ref_user_dict = format_reference_response(resp=ml_ref_user_resp, function_name='contextUserInfo')

In [56]:
ml_ref_user_dict

{'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': {'name': 'Venkata Dikshit',
  'keywords': ['takeover of the conversation',
   'type tag',
   'Pretty color zoom',
   'test case',
   'list all the reasons',
   'colander Production Office',
   'create a distance Matrix Paris decision',
   'model that can detect employee benefits',
   'fact that context',
   'aggregate Docker',
   'presented entry point',
   'hard disk',
   'second in the action',
   'medium market yep',
   'slight fine tuning',
   'potential action item',
   'big change',
   'entities between the',
   'continue generating the groups',
   'kind of separation',
   'community uploads',
   'Philip sentences',
   'feel like the lsdm',
   'training Data customer',
   'Graham',
   'ten a second',
   'link of that example',
   'grammar rules',
   'Ram three four sentences',
   'yesterday sentences',
   'expect it to behave',
   'check for this',
   'Downstream dances',
   'killed immunities',
   'capture the context',
   'docker perspec

In [73]:
len(ml_ref_user_dict)

8

In [74]:
from copy import deepcopy
new_ref_user_dict = deepcopy(ref_user_dict)
for user_id, it in ml_ref_user_dict.items():
    name = it["name"]
    ml_keyword_list = it["keywords"]
    ml_seg_ids = it["segment_id"]
    ml_seg_texts = it["text"]
    if user_id in ref_user_dict.keys():
        keyword_list = ref_user_dict[user_id]["keywords"]
        keyword_list.extend(ml_keyword_list)
        seg_id = ref_user_dict[user_id]["segment_id"]
        segment_text = ref_user_dict[user_id]["text"]
        seg_id.extend(ml_seg_ids)
        segment_text.extend(ml_seg_texts)
        
        new_ref_user_dict[user_id]["keywords"] = keyword_list
        new_ref_user_dict[user_id]["segment_id"] = seg_id
        new_ref_user_dict[user_id]["text"] = segment_text
    else:
        new_ref_user_dict.update({
            user_id: {
                "name": name,
                "keywords": ml_keyword_list,
                "segment_id": ml_seg_ids,
                "text": ml_seg_texts
            }
        })

In [75]:
len(new_ref_user_dict)

15

In [76]:
for k, v in new_ref_user_dict.items():
    print(v["name"])

Parshwa Nemi Jain
Vamshi Krishna
mithun
Nisha Yadav
Franklin Ferdinand
Shashank
Trishanth Diwate
Deep Moradia
Venkata Dikshit
Vani
Reagan Rewop
Karthik Muralidharan
Shubham
Krishna Sai
Arjun Kini


In [77]:
len(new_ref_user_dict)

15

In [78]:
to_json(new_ref_user_dict, filename="reference_prod_user")

In [99]:
ref_user_text_dict = {k: new_ref_user_dict[k]["text"] for k in new_ref_user_dict.keys()}

In [100]:
len(new_ref_user_dict)

14

In [79]:
new_ref_user_dict['2c944512-17a0-4912-9a16-6a3408da807c']['name']

'Franklin Ferdinand'

In [80]:
del new_ref_user_dict['2c944512-17a0-4912-9a16-6a3408da807c']

## Initiate Lambda function

In [88]:
from boto3 import client
from botocore.client import Config

In [89]:
aws_config = Config(
        connect_timeout=180,
        read_timeout=300,
        retries={"max_attempts": 2},
        region_name="us-east-1",
    )

In [90]:
lambda_client = client("lambda", config=aws_config)

In [92]:
vec = Vectorizer(lambda_client, "sentence-encoder-lambda")

## Run LSH on reference data

In [101]:
us = UserSearch(input_dict=ref_user_text_dict, vectorizer=vec, user_vector_data=None, num_buckets=8, hash_size=4)

In [102]:
us.featurize(write=True, file_name="reference_user_text_vector.pickle")

In [103]:
us.num_features_in_input

{'0bbbfe84-c661-45af-8d0f-fcd5258bba38': 41,
 '1a215425-8449-4fca-ba95-7d768b595b80': 41,
 '84fbaa66-a247-4ea2-9ae0-53f3a2e519d6': 113,
 'c66797a9-2e6d-46ad-9573-926e57f7dac3': 21,
 '7e7ccbba-232d-411a-a95a-d3f244a35f40': 160,
 '75bdf310-110b-4b8f-ab88-b16fafce920e': 58,
 'b4a57b25-de68-446c-ac99-0f856d3fe4d5': 58,
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': 160,
 '65bb8395-2fb5-4409-a4bb-59bb707f1375': 58,
 'fb52cb66-3aec-4795-aee3-8ccfd904d315': 154,
 '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368': 49,
 'ecfeeb75-7f0a-4d47-af1e-bd513929264a': 117,
 '8d6db5f7-d9b7-4c54-ba38-fe710ffcaf3f': 57,
 '81a3e154-6937-4fce-ba1c-f972faa209b2': 173}

In [104]:
alt_keywords = ['Google Calendar', 
                'Platform Mitten', 
                'GPU', 
                'API changes yesterday', 
                'ICS', 
                'topics in a structured format', 
                'Golang', 
                'parse this ICS message', 
                'store structured data', 
                'occur then the meeting URL']

In [115]:
user_keywords = ref_user_dict['b1e8787a-9a1f-4859-ac11-cbb6a8124fd9'].get('keywords')

In [135]:
eg_text = ref_user_dict['b1e8787a-9a1f-4859-ac11-cbb6a8124fd9'].get('text')[0]

In [116]:
result = us.query(kw_list=user_keywords)

num results 2696017


In [117]:
for r in sorted(result, key=result.get, reverse=True):
    print(r, new_ref_user_dict[r]["name"], result[r])

ecfeeb75-7f0a-4d47-af1e-bd513929264a Shubham 2309.3076923076924
84fbaa66-a247-4ea2-9ae0-53f3a2e519d6 mithun 2233.1769911504425
62b6ae1d-7f83-4b0b-b205-5f7c72bc3368 Karthik Muralidharan 2208.7551020408164
fb52cb66-3aec-4795-aee3-8ccfd904d315 Reagan Rewop 2133.5779220779223
81a3e154-6937-4fce-ba1c-f972faa209b2 Arjun Kini 2124.624277456647
7e7ccbba-232d-411a-a95a-d3f244a35f40 Shashank 2122.60625
b1e8787a-9a1f-4859-ac11-cbb6a8124fd9 Venkata Dikshit 2122.60625
c66797a9-2e6d-46ad-9573-926e57f7dac3 Nisha Yadav 2111.190476190476
75bdf310-110b-4b8f-ab88-b16fafce920e Trishanth Diwate 2108.5689655172414
b4a57b25-de68-446c-ac99-0f856d3fe4d5 Deep Moradia 2108.5689655172414
65bb8395-2fb5-4409-a4bb-59bb707f1375 Vani 2108.5689655172414
8d6db5f7-d9b7-4c54-ba38-fe710ffcaf3f Krishna Sai 2042.4035087719299
0bbbfe84-c661-45af-8d0f-fcd5258bba38 Parshwa Nemi Jain 1978.560975609756
1a215425-8449-4fca-ba95-7d768b595b80 Vamshi Krishna 1978.560975609756


In [118]:
m_result = [result[r] for r in result]

In [119]:
np.mean(m_result)

2120.791236983369

## Ablation tests

In [508]:
num_buckets_list = [2, 4, 6, 8, 10, 12, 14]
hash_size_list = [3, 4, 6, 8, 10, 12, 15, 20]

In [511]:
for h in hash_size_list:
    for num in num_buckets_list:
        print()
        print("Bucket size: {}, Hash size: {}".format(num, h))
        print()
        us = UserSearch(input_dict=ref_user_kw_dict, vectorizer=Vectorizer(), user_vector_data=user_vectors, num_buckets=num, hash_size=h)
        us.featurize(write=False)
        result = us.query(kw_list=alt_keywords)
        for r in sorted(result, key=result.get, reverse=True):
            print(r, new_ref_user_dict[r]["name"], result[r])


Bucket size: 2, Hash size: 3

num results 49803
0bbbfe84-c661-45af-8d0f-fcd5258bba38 Parshwa Nemi Jain 5.231454005934718
b4a57b25-de68-446c-ac99-0f856d3fe4d5 Deep Moradia 5.154109589041096
84fbaa66-a247-4ea2-9ae0-53f3a2e519d6 mithun 5.1019736842105265
65bb8395-2fb5-4409-a4bb-59bb707f1375 Vani 5.061728395061729
62b6ae1d-7f83-4b0b-b205-5f7c72bc3368 Karthik Muralidharan 4.983778625954199
7e7ccbba-232d-411a-a95a-d3f244a35f40 Shashank 4.975395430579965
fb52cb66-3aec-4795-aee3-8ccfd904d315 Reagan Rewop 4.954500494559841
81a3e154-6937-4fce-ba1c-f972faa209b2 Arjun Kini 4.954441913439635
c66797a9-2e6d-46ad-9573-926e57f7dac3 Nisha Yadav 4.944700460829493
ecfeeb75-7f0a-4d47-af1e-bd513929264a Shubham 4.8497109826589595
75bdf310-110b-4b8f-ab88-b16fafce920e Trishanth Diwate 4.8431372549019605
b1e8787a-9a1f-4859-ac11-cbb6a8124fd9 Venkata Dikshit 4.775021385799829
8d6db5f7-d9b7-4c54-ba38-fe710ffcaf3f Krishna Sai 4.701342281879195
1a215425-8449-4fca-ba95-7d768b595b80 Vamshi Krishna 4.687103594080338



## Filteration

In [125]:
cum_result_score = np.sum([result[r] for r in result])
mean_norm_result_score = np.mean([result[r]/cum_result_score for r in result])

In [126]:
mean_norm_result_score

0.07142857142857142

In [127]:
similar_users = {k: result[k]/cum_result_score for k in sorted(result, key=result.get, reverse=True) if result[k]/cum_result_score >= mean_norm_result_score}

In [128]:
similar_users

{'ecfeeb75-7f0a-4d47-af1e-bd513929264a': 0.07777783431676966,
 '84fbaa66-a247-4ea2-9ae0-53f3a2e519d6': 0.0752137407225068,
 '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368': 0.07439120778278717,
 'fb52cb66-3aec-4795-aee3-8ccfd904d315': 0.07185922892737838,
 '81a3e154-6937-4fce-ba1c-f972faa209b2': 0.07155766881470718,
 '7e7ccbba-232d-411a-a95a-d3f244a35f40': 0.07148970134303044,
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': 0.07148970134303044}

In [129]:
similar_users_kw_dict = {k: new_ref_user_dict[k]["keywords"] for k in similar_users.keys()}

In [130]:
similar_users_kw_list = list(set([words for user_kw in similar_users_kw_dict.values() for words in user_kw]))

In [131]:
similar_users_kw_list

['data etcetera',
 'query for feature',
 'call to parallel calls',
 'bit signal',
 'dirty demo',
 'shape we can test',
 'change what they send',
 'straightforward to load multiple graph data',
 'simple sample limit',
 'ten certain marker',
 'model remove',
 'model that can detect employee benefits',
 'create a distance Matrix Paris decision',
 'purchase those sentences',
 'ten minutes',
 'green go the black phone addition',
 'presented entry point',
 'hard disk',
 'part of your final vendor',
 'course the old summary approach',
 'second in the action',
 'negative impact feature',
 'slight fine tuning',
 'current is so yesterday',
 'tests called Sandman',
 'identifiers and assigns',
 'start amazing activities',
 'basic validations',
 'Entity Graph',
 'multiple callbacks',
 'map all the tags',
 "Ether Barton's",
 'Nothing',
 'optimal about the computation',
 'good sleep',
 'main comparison',
 'instance ID mind',
 'Graham',
 'change to print',
 'domains groups',
 'testing all the pipeline

In [132]:
ws = WordSearch(input_list=similar_users_kw_list, vectorizer=vec, num_buckets=8)

In [133]:
ws.featurize()

In [138]:
sim_keyword_result = ws.query(kw_list=eg_text)

In [140]:
eg_text

"Yeah, so the last night I was like finally I was able to populate the entire production data 2D graph. So now I'm going to use just our engineering Channel and Emily a channel user keywords to to test the recommendation. So previously before before this I tested it wandered Watchers on using staging to dance so I I was testing It on stage in two hours or so. But the data was cute, so I was not able to give any conclusive results then so right now the idea is with our engineering Channel and Emily a channel. We have a good number of keywords representing each of the users. For example, like for user session related words, like key phrases and graph and towards things for others. There are a language model. So hopefully it so if we're going by the last class times Bowman goes with wanted watches the similarity search and comparison is working fine. So if that if that is actually working fine, then we should be able to see good recommended watches based on today's test. "

In [139]:
for sw in sorted(sim_keyword_result, key=sim_keyword_result.get, reverse=True):
    if sw not in alt_keywords:
        for user_id, kw_list in similar_users_kw_dict.items():
            if sw in kw_list:
                print(sw, new_ref_user_dict[user_id]["name"], sim_keyword_result[sw])

events stable Karthik Muralidharan 0.2578125
structure for the minds Karthik Muralidharan 0.193359375
publish an event saying group Shashank 0.193359375
pixels minds Venkata Dikshit 0.193359375
Security my whole idea Venkata Dikshit 0.193359375
stigma conditional Karthik Muralidharan 0.193359375
community based its decision Venkata Dikshit 0.193359375
idea for comparison Reagan Rewop 0.193359375
idea for comparison Arjun Kini 0.193359375
idea for comparison Venkata Dikshit 0.193359375
proper cosine similarity match Shashank 0.193359375
model performance Venkata Dikshit 0.193359375
recording some other metadata Karthik Muralidharan 0.193359375
feature vector Shashank 0.193359375
trend model Shubham 0.193359375
trend model Reagan Rewop 0.193359375
trend model Arjun Kini 0.193359375
trend model Venkata Dikshit 0.193359375
feature Vector Reagan Rewop 0.193359375
feature Vector Arjun Kini 0.193359375
feature Vector Venkata Dikshit 0.193359375
video hundred thousand feature vectors Shubham 0

In [141]:
all_similar_user_freq_list = []
similar_user_freq_list = []
mean_word_sim = np.mean([sim_keyword_result[r] for r in sim_keyword_result])
for sw in sorted(sim_keyword_result, key=sim_keyword_result.get, reverse=True):
    if sw not in alt_keywords:
        for user_id, kw_list in similar_users_kw_dict.items():
            if sw in kw_list:
                all_similar_user_freq_list.append(user_id)
                if sim_keyword_result[sw] >= mean_word_sim:
                    similar_user_freq_list.append(user_id)
                # print(sw, new_ref_user_dict[user_id]["name"], sim_keyword_result[sw])

In [142]:
similar_user_freq_list

['62b6ae1d-7f83-4b0b-b205-5f7c72bc3368',
 '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368',
 '7e7ccbba-232d-411a-a95a-d3f244a35f40',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 'fb52cb66-3aec-4795-aee3-8ccfd904d315',
 '81a3e154-6937-4fce-ba1c-f972faa209b2',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 '7e7ccbba-232d-411a-a95a-d3f244a35f40',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 '62b6ae1d-7f83-4b0b-b205-5f7c72bc3368',
 '7e7ccbba-232d-411a-a95a-d3f244a35f40',
 'ecfeeb75-7f0a-4d47-af1e-bd513929264a',
 'fb52cb66-3aec-4795-aee3-8ccfd904d315',
 '81a3e154-6937-4fce-ba1c-f972faa209b2',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 'fb52cb66-3aec-4795-aee3-8ccfd904d315',
 '81a3e154-6937-4fce-ba1c-f972faa209b2',
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9',
 'ecfeeb75-7f0a-4d47-af1e-bd513929264a',
 'fb52cb66-3aec-4795-aee3-8ccfd904d315',
 '81a3e154-6937-4fce-ba1c-f972faa209b2',
 'b1e8787a-9a1f-

In [143]:
from collections import Counter

In [144]:
af = Counter(all_similar_user_freq_list)

In [145]:
af

Counter({'62b6ae1d-7f83-4b0b-b205-5f7c72bc3368': 440,
         '7e7ccbba-232d-411a-a95a-d3f244a35f40': 371,
         'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': 805,
         'fb52cb66-3aec-4795-aee3-8ccfd904d315': 293,
         '81a3e154-6937-4fce-ba1c-f972faa209b2': 191,
         'ecfeeb75-7f0a-4d47-af1e-bd513929264a': 97,
         '84fbaa66-a247-4ea2-9ae0-53f3a2e519d6': 128})

In [146]:
f = Counter(similar_user_freq_list)

In [147]:
f

Counter({'62b6ae1d-7f83-4b0b-b205-5f7c72bc3368': 72,
         '7e7ccbba-232d-411a-a95a-d3f244a35f40': 61,
         'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': 152,
         'fb52cb66-3aec-4795-aee3-8ccfd904d315': 65,
         '81a3e154-6937-4fce-ba1c-f972faa209b2': 40,
         'ecfeeb75-7f0a-4d47-af1e-bd513929264a': 18,
         '84fbaa66-a247-4ea2-9ae0-53f3a2e519d6': 17})

In [148]:
total_res = np.sum([s for s in af.values()])
total_res = 1
final_counter = {u: (f[u]/af[u])*total_res for u in af.keys()}

    

In [149]:
final_counter

{'62b6ae1d-7f83-4b0b-b205-5f7c72bc3368': 0.16363636363636364,
 '7e7ccbba-232d-411a-a95a-d3f244a35f40': 0.16442048517520216,
 'b1e8787a-9a1f-4859-ac11-cbb6a8124fd9': 0.18881987577639753,
 'fb52cb66-3aec-4795-aee3-8ccfd904d315': 0.22184300341296928,
 '81a3e154-6937-4fce-ba1c-f972faa209b2': 0.2094240837696335,
 'ecfeeb75-7f0a-4d47-af1e-bd513929264a': 0.18556701030927836,
 '84fbaa66-a247-4ea2-9ae0-53f3a2e519d6': 0.1328125}

In [151]:
new_ref_user_dict['fb52cb66-3aec-4795-aee3-8ccfd904d315']["name"]

'Reagan Rewop'

In [152]:
for u_id, score in final_counter.items():
    print(new_ref_user_dict[u_id]["name"], score)

Karthik Muralidharan 0.16363636363636364
Shashank 0.16442048517520216
Venkata Dikshit 0.18881987577639753
Reagan Rewop 0.22184300341296928
Arjun Kini 0.2094240837696335
Shubham 0.18556701030927836
mithun 0.1328125


## Explainability

In [153]:
from nltk import word_tokenize, pos_tag, sent_tokenize

In [154]:
pos_list= ['NN', 'NNS', 'NNP']

In [155]:
pos_ = ['NN', 'NNS']

In [156]:
pos_ in pos_list

False

In [507]:
for sw in sorted(sim_keyword_result, key=sim_keyword_result.get, reverse=True):
    if sw not in alt_keywords:
        for user_id, kw_list in similar_users_kw_dict.items():
            if sw in kw_list:
                print(sw, new_ref_user_dict[user_id]["name"], sim_keyword_result[sw], pos_tag(word_tokenize(sw)))

golang parser Trishanth Diwate 0.0234375 [('golang', 'NN'), ('parser', 'NN')]
offline queries Reagan Rewop 0.021484375 [('offline', 'NN'), ('queries', 'NNS')]
API service Reagan Rewop 0.021484375 [('API', 'NNP'), ('service', 'NN')]
current schema Shashank 0.021484375 [('current', 'JJ'), ('schema', 'NN')]
slack with two sections mithun 0.01953125 [('slack', 'NN'), ('with', 'IN'), ('two', 'CD'), ('sections', 'NNS')]
publish an event saying group Shashank 0.01953125 [('publish', 'VB'), ('an', 'DT'), ('event', 'NN'), ('saying', 'VBG'), ('group', 'NN')]
start calling Lambda Shashank 0.01953125 [('start', 'NN'), ('calling', 'VBG'), ('Lambda', 'NNP')]
work different Lambda Shashank 0.01953125 [('work', 'NN'), ('different', 'JJ'), ('Lambda', 'NNP')]
expose apis mithun 0.017578125 [('expose', 'RB'), ('apis', 'NN')]
websocket message mithun 0.017578125 [('websocket', 'NN'), ('message', 'NN')]
second the API mithun 0.017578125 [('second', 'JJ'), ('the', 'DT'), ('API', 'NNP')]
preference to organi

In [157]:
top_words = []
for sw in sorted(sim_keyword_result, key=sim_keyword_result.get, reverse=True):
    if sw not in alt_keywords:
        top_words.append(sw)

In [158]:
top_words

['events stable',
 'structure for the minds',
 'publish an event saying group',
 'pixels minds',
 'Security my whole idea',
 'stigma conditional',
 'community based its decision',
 'idea for comparison',
 'proper cosine similarity match',
 'model performance',
 'recording some other metadata',
 'feature vector',
 'trend model',
 'feature Vector',
 'video hundred thousand feature vectors',
 'segments can call the entity extraction Lambda identities',
 'group recognition',
 'kind of around testing calls',
 'beta graphs',
 'groups segments',
 'Lambda to promote',
 'topic group',
 'Mind versus model discussion size',
 'map all these feature vectors',
 'release models',
 'phone after of fifty ten minutes till',
 'pins are really bad',
 'remove attendee',
 'create because communities community algorithms',
 'simple sample limit',
 'green go the black phone addition',
 'Weather nice',
 'transfer topics',
 'start like basic display',
 'process into a lambda function kind',
 'planet and then pu

In [179]:
top_10 = top_words[:500]

In [180]:
def filter_pos(word_list):
    filtered_word = []
    
    for word in word_list:
        pos_word = pos_tag(word_tokenize(word))
        counter = 0
        for tags in pos_word:
            p = tags[1]
            if p in pos_list:
                counter += 1

        if counter == len(word_tokenize(word)):
            filtered_word.append(word)
    return filtered_word

In [181]:
filtered_list = filter_pos(top_10)
filtered_list

['pixels minds',
 'stigma conditional',
 'model performance',
 'feature vector',
 'trend model',
 'feature Vector',
 'group recognition',
 'beta graphs',
 'groups segments',
 'topic group',
 'Mind versus model discussion size',
 'release models',
 'Weather nice',
 'transfer topics',
 'event call life',
 'Olympics',
 'software engineering mind',
 'entity recognition model',
 'perform population',
 'topics ios',
 'test calls',
 'immune system',
 'entity features',
 'audio problems',
 'Mask language model',
 'contacts ideas',
 'highlighter topics',
 'start feature',
 'Temple',
 'model service industry',
 'mind generation',
 'media drug tests',
 'entity recognition Lambda',
 'video quality',
 'mask language model',
 'action detection',
 'client libraries',
 'crowd structure',
 'mapping uuid',
 'intent classification service',
 'mind generate data collection',
 'mesh screen',
 'instance ID mind',
 'Neptune',
 'feature extraction phrase face',
 'channel ID playlist',
 'topic detection',
 'fe

In [172]:
!pip install fuzzywuzzy

In [173]:
from fuzzywuzzy import process, fuzz

/Users/shashank/anaconda3/envs/entity-graph/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [182]:
list(process.dedupe(filtered_list))

['pixels minds',
 'stigma conditional',
 'model performance',
 'feature Vector',
 'trend model',
 'entity recognition Lambda',
 'entity graphs',
 'groups segments',
 'topic group',
 'Mind versus model discussion size',
 'release models',
 'Weather nice',
 'transfer topics',
 'event call life',
 'Olympics',
 'software engineering mind',
 'Standalone graph population service',
 'highlighter topics',
 'test calls',
 'immune system',
 'entity features',
 'deadline design problems',
 'Mask language model',
 'contacts ideas',
 'Temple',
 'model service industry',
 'mind generation',
 'media drug tests',
 'video quality',
 'action detection',
 'client libraries',
 'crowd structure',
 'mapping uuid',
 'intent classification service',
 'mind generate data collection',
 'mesh screen',
 'instance ID mind',
 'Neptune',
 'feature extraction phrase face',
 'channel ID playlist',
 'topic segregation',
 'feature space response',
 'reason record',
 'sentence endings',
 'mirror defense',
 'Shri Arjun',


In [185]:
eg_text

"Yeah, so the last night I was like finally I was able to populate the entire production data 2D graph. So now I'm going to use just our engineering Channel and Emily a channel user keywords to to test the recommendation. So previously before before this I tested it wandered Watchers on using staging to dance so I I was testing It on stage in two hours or so. But the data was cute, so I was not able to give any conclusive results then so right now the idea is with our engineering Channel and Emily a channel. We have a good number of keywords representing each of the users. For example, like for user session related words, like key phrases and graph and towards things for others. There are a language model. So hopefully it so if we're going by the last class times Bowman goes with wanted watches the similarity search and comparison is working fine. So if that if that is actually working fine, then we should be able to see good recommended watches based on today's test. "

In [184]:
process.extractBests(eg_text, filtered_list, limit=20)

[('model performance', 57),
 ('trend model', 57),
 ('Mind versus model discussion size', 57),
 ('software engineering mind', 57),
 ('entity recognition model', 57),
 ('test calls', 57),
 ('Mask language model', 57),
 ('model service industry', 57),
 ('mask language model', 57),
 ('mind generate data collection', 57),
 ('channel ID playlist', 57),
 ('Standalone graph population service', 57),
 ('graph today', 57),
 ('entity graph service', 57),
 ('graph data', 57),
 ('test streams', 57),
 ('Easter engineering Gingka', 57),
 ('bird base model', 57),
 ('rest right HR machine learning', 57),
 ('production files', 57)]